In [104]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [105]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [106]:
data =data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [107]:
#encode categorical variable

label_gender= LabelEncoder()
data['Gender']=label_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [108]:
#one hot encoding "geography "
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder=OneHotEncoder()
geo_encoder=one_hot_encoder.fit_transform(data[['Geography']])
geo_encoder

<10000x3 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [109]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [110]:
geo_encoder_df= pd.DataFrame(geo_encoder.toarray(),columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [111]:
#combine 
data = pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [112]:
#save the encoder and scaler 

with open ('encoder_gender.pkl','wb') as file:
    pickle.dump(label_gender,file)
    
with open('one_hot_encoder', 'wb') as file:
    pickle.dump(one_hot_encoder,file)   
    

In [113]:
#divide and scaler 
X=data.drop('Exited',axis=1)
y=data['Exited']

X_train, X_test, ytrain, ytest= train_test_split(X,y,test_size=2,random_state=42)
scale = StandardScaler()
X_train=scale.fit_transform(X_train)
X_test=scale.transform(X_test)

In [114]:
X_train

array([[-0.51250729, -1.09578666,  0.48416327, ..., -1.00280449,
        -0.57865919,  1.74250804],
       [-1.49537231,  0.9125864 ,  1.91436606, ..., -1.00280449,
         1.72813292, -0.57388544],
       [-0.93669114, -1.09578666, -1.13673323, ..., -1.00280449,
        -0.57865919,  1.74250804],
       ...,
       [ 0.8738497 , -1.09578666, -0.08791785, ...,  0.99720335,
        -0.57865919, -0.57388544],
       [ 0.17032526,  0.9125864 ,  0.38881642, ...,  0.99720335,
        -0.57865919, -0.57388544],
       [ 0.48070369,  0.9125864 ,  1.15159124, ..., -1.00280449,
         1.72813292, -0.57388544]])

In [115]:
with open('scale', 'wb') as file:
    pickle.dump(scale,file) 

In [116]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [117]:
X_train.shape[1]

X_train

array([[-0.51250729, -1.09578666,  0.48416327, ..., -1.00280449,
        -0.57865919,  1.74250804],
       [-1.49537231,  0.9125864 ,  1.91436606, ..., -1.00280449,
         1.72813292, -0.57388544],
       [-0.93669114, -1.09578666, -1.13673323, ..., -1.00280449,
        -0.57865919,  1.74250804],
       ...,
       [ 0.8738497 , -1.09578666, -0.08791785, ...,  0.99720335,
        -0.57865919, -0.57388544],
       [ 0.17032526,  0.9125864 ,  0.38881642, ...,  0.99720335,
        -0.57865919, -0.57388544],
       [ 0.48070369,  0.9125864 ,  1.15159124, ..., -1.00280449,
         1.72813292, -0.57388544]])

In [118]:


!pip install tensorflow

In [119]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [120]:
#build an Ann Model 

model= Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #hidden layer 1
    Dense(32,activation='relu'),# hidden layer 2
    Dense(1,activation="sigmoid") #output layer
    
    
])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [121]:
model.summary()

Model: "sequential_45"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_135 (Dense)               │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_136 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_137 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [125]:
#loss and opt 
import tensorflow

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss= tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x307cc3ce0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [126]:
#compile the model
model.compile(optimizer=opt,loss="BinaryCrossentropy",metrics=['accuracy'])

In [127]:
#set up the tensorboard 

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/"+datetime.datetime.now().strftime("%d/%m/%Y")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)




In [130]:
#set up the Early stopping 
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [132]:
### TRAIN THE MODEL 
history = model.fit(
    X_train,ytrain,validation_data=(X_test,ytest),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8186 - loss: 0.4282 - val_accuracy: 1.0000 - val_loss: 0.0344
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 736us/step - accuracy: 0.8577 - loss: 0.3534 - val_accuracy: 1.0000 - val_loss: 0.0640
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 675us/step - accuracy: 0.8557 - loss: 0.3520 - val_accuracy: 1.0000 - val_loss: 0.0543
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 754us/step - accuracy: 0.8604 - loss: 0.3333 - val_accuracy: 1.0000 - val_loss: 0.0375
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - accuracy: 0.8640 - loss: 0.3351 - val_accuracy: 1.0000 - val_loss: 0.0463
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 844us/step - accuracy: 0.8617 - loss: 0.3399 - val_accuracy: 1.0000 - val_loss: 0.0291
Epoch 7/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 712us/step - accuracy: 0.8578 - loss: 0.3370 - val_accuracy: 1.0000 - val_loss: 0.0639
Epoch 8/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 677us/step - accuracy: 0.8695 - loss: 0.3

In [133]:
model.save('model.h5')

In [143]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [145]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 8618), started 0:01:07 ago. (Use '!kill 8618' to kill it.)